# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 24 2022 8:30AM,240360,10,0085798384,ISDIN Corporation,Released
1,Jun 24 2022 8:30AM,240364,10,0085798385,ISDIN Corporation,Released
2,Jun 24 2022 8:30AM,240364,10,0085798386,ISDIN Corporation,Released
3,Jun 24 2022 8:30AM,240365,10,0085798394,ISDIN Corporation,Released
4,Jun 24 2022 8:30AM,240360,10,0085798389,ISDIN Corporation,Released
5,Jun 24 2022 8:30AM,240365,10,0085798390,ISDIN Corporation,Released
6,Jun 24 2022 8:30AM,240365,10,0085798393,ISDIN Corporation,Released
7,Jun 24 2022 8:30AM,240365,10,0085798396,ISDIN Corporation,Released
8,Jun 24 2022 8:30AM,240365,10,0085798399,ISDIN Corporation,Released
9,Jun 24 2022 8:30AM,240361,10,0085798397,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,240360,Released,5
26,240361,Released,1
27,240363,Released,22
28,240364,Released,9
29,240365,Released,29


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240360,NaN,5.0
240361,NaN,1.0
240363,NaN,22.0
240364,NaN,9.0
240365,NaN,29.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240283,1.0,0.0
240285,0.0,1.0
240295,0.0,1.0
240311,1.0,1.0
240312,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240283,1,0
240285,0,1
240295,0,1
240311,1,1
240312,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240283,1,0
1,240285,0,1
2,240295,0,1
3,240311,1,1
4,240312,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240283,1,
1,240285,,1
2,240295,,1
3,240311,1,1
4,240312,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 24 2022 8:30AM,240360,10,ISDIN Corporation
1,Jun 24 2022 8:30AM,240364,10,ISDIN Corporation
3,Jun 24 2022 8:30AM,240365,10,ISDIN Corporation
9,Jun 24 2022 8:30AM,240361,10,ISDIN Corporation
44,Jun 23 2022 5:07PM,240363,20,Alumier Labs Inc.
66,Jun 23 2022 4:56PM,240359,12,LF of America Corp.
67,Jun 23 2022 4:18PM,240357,16,Sartorius Bioprocess Solutions
70,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.
71,Jun 23 2022 4:03PM,240354,12,"NBTY Global, Inc."
73,Jun 23 2022 3:52PM,240352,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 24 2022 8:30AM,240360,10,ISDIN Corporation,,5
1,Jun 24 2022 8:30AM,240364,10,ISDIN Corporation,,9
2,Jun 24 2022 8:30AM,240365,10,ISDIN Corporation,,29
3,Jun 24 2022 8:30AM,240361,10,ISDIN Corporation,,1
4,Jun 23 2022 5:07PM,240363,20,Alumier Labs Inc.,,22
5,Jun 23 2022 4:56PM,240359,12,LF of America Corp.,,1
6,Jun 23 2022 4:18PM,240357,16,Sartorius Bioprocess Solutions,,3
7,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.,,1
8,Jun 23 2022 4:03PM,240354,12,"NBTY Global, Inc.",1,1
9,Jun 23 2022 3:52PM,240352,16,Sartorius Stedim Filters Inc.,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 8:30AM,240360,10,ISDIN Corporation,5,
1,Jun 24 2022 8:30AM,240364,10,ISDIN Corporation,9,
2,Jun 24 2022 8:30AM,240365,10,ISDIN Corporation,29,
3,Jun 24 2022 8:30AM,240361,10,ISDIN Corporation,1,
4,Jun 23 2022 5:07PM,240363,20,Alumier Labs Inc.,22,
5,Jun 23 2022 4:56PM,240359,12,LF of America Corp.,1,
6,Jun 23 2022 4:18PM,240357,16,Sartorius Bioprocess Solutions,3,
7,Jun 23 2022 4:09PM,240355,16,Sartorius Stedim Filters Inc.,1,
8,Jun 23 2022 4:03PM,240354,12,"NBTY Global, Inc.",1,1
9,Jun 23 2022 3:52PM,240352,16,Sartorius Stedim Filters Inc.,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 8:30AM,240360,10,ISDIN Corporation,5,
1,Jun 24 2022 8:30AM,240364,10,ISDIN Corporation,9,
2,Jun 24 2022 8:30AM,240365,10,ISDIN Corporation,29,
3,Jun 24 2022 8:30AM,240361,10,ISDIN Corporation,1,
4,Jun 23 2022 5:07PM,240363,20,Alumier Labs Inc.,22,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 8:30AM,240360,10,ISDIN Corporation,5.0,NaN
1,Jun 24 2022 8:30AM,240364,10,ISDIN Corporation,9.0,NaN
2,Jun 24 2022 8:30AM,240365,10,ISDIN Corporation,29.0,NaN
3,Jun 24 2022 8:30AM,240361,10,ISDIN Corporation,1.0,NaN
4,Jun 23 2022 5:07PM,240363,20,Alumier Labs Inc.,22.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 24 2022 8:30AM,240360,10,ISDIN Corporation,5.0,0.0
1,Jun 24 2022 8:30AM,240364,10,ISDIN Corporation,9.0,0.0
2,Jun 24 2022 8:30AM,240365,10,ISDIN Corporation,29.0,0.0
3,Jun 24 2022 8:30AM,240361,10,ISDIN Corporation,1.0,0.0
4,Jun 23 2022 5:07PM,240363,20,Alumier Labs Inc.,22.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1682391,46.0,1.0
12,720996,2.0,2.0
15,480684,25.0,0.0
16,1682386,9.0,0.0
19,480620,2.0,0.0
20,961350,24.0,2.0
21,240295,1.0,0.0
22,480672,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1682391,46.0,1.0
1,12,720996,2.0,2.0
2,15,480684,25.0,0.0
3,16,1682386,9.0,0.0
4,19,480620,2.0,0.0
5,20,961350,24.0,2.0
6,21,240295,1.0,0.0
7,22,480672,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,46.0,1.0
1,12,2.0,2.0
2,15,25.0,0.0
3,16,9.0,0.0
4,19,2.0,0.0
5,20,24.0,2.0
6,21,1.0,0.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,46.0
1,12,Released,2.0
2,15,Released,25.0
3,16,Released,9.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
Released,46.0,2.0,25.0,9.0,2.0,24.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Released,46.0,2.0,25.0,9.0,2.0,24.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,1.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Released,46.0,2.0,25.0,9.0,2.0,24.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()